<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_HR_gr1/blob/main/Kiparenko%20Nikolai/INSPECTRUM_CLINIC_HR_RECSYS_GR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# Работа с массивами данных
import numpy as np 

# Работа с таблицами
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation 

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Заполнение последовательностей до определенной длины
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Кодирование тестовых меток
from sklearn.preprocessing import LabelEncoder

# Разбиение на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split

# автоматически парсит строку и сохранит ее в переменной datetime
import datetime

# Загрузка датасетов из облака google
import gdown

# Функции операционной системы
import os

# Работа со временем
import time

# Регулярные выражения
import re

# Запись в файлы и чтение из файлов структур данных Python
import pickle

# Отрисовка графиков
import matplotlib.pyplot as plt


%matplotlib inline

In [2]:
import gdown                                      # Подключим функцию gdown
gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_HR/datasets/%D0%94%D0%B0%D1%82%D0%B0%D0%A1%D0%B5%D1%82%20%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20581%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.xlsx','dataS.xlsx', quiet=True)   

'dataS.xlsx'

In [197]:
data = pd.read_excel (r'dataS.xlsx', index_col=0)
data.max_columns = 30
data.columns = ['Response_date', 'Full_name', 'Gender', 'Salary', 'Job', 'City', 'Employment', 'Schedule', 'Experience', 'Last_place', 'Position', 'Education', 'Update_resume', 'Suitable']

In [153]:
data.head(5)

,Response_date,Full_name,Gender,Salary,Job,City,Employment,Schedule,Experience,Last_place,Position,Education,Update_resume,Suitable
0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,2023-02-27 00:00:00,Фролов Артур Михайлович,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да
3,2023-01-11 00:00:00,Хозаев Расул Шагаманович,"Мужчина, 32 года, родился 7 декабря 1990",не указана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да
4,2023-01-11 00:00:00,Шадрин Олег Михайлович,"Мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да


In [198]:
month_name = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня',
      7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}


In [199]:
Birth = data['Gender'].apply(lambda x: x.split(',')[2].strip() if len(x.split(','))>2 else '')
data = data.assign(Birth=Birth)

Age = data['Gender'].apply(lambda x: x.split(',')[1].strip() if len(x.split(','))>1 else '')
data = data.assign(Age=Age)


In [200]:
for i, row in data.iterrows():
  #Gender
  row.Gender = row.Gender.split(',')[0].strip() 
  #Женщина = 0, Мужчина = 1
  row.Gender = row.Gender.replace('Женщина','0')
  row.Gender = row.Gender.replace('Мужчина','1')

  #Birth
  row.Birth = row.Birth.replace('родился','')
  row.Birth = row.Birth.replace('родилась','')
 
  #Age
  row.Age = row.Age.replace('года','')
  row.Age = row.Age.replace('год','')
  row.Age = row.Age.replace('лет','')
  row.Age = row.Age='' if 0 else row.Age

  #Salary
  row.Salary = row.Salary.replace(' ','') 
  row.Salary = str(float(row.Salary.replace('KGS',''))*0.89)      if row.Salary.find('KGS') != -1 else row.Salary
  row.Salary = str(float(row.Salary.replace('KZT',''))*0.18)      if row.Salary.find('KZT') != -1 else row.Salary
  row.Salary = str(float(row.Salary.replace('USD',''))*78.00)     if row.Salary.find('USD') != -1 else row.Salary
  row.Salary = str(float(row.Salary.replace('EUR',''))*85.00)     if row.Salary.find('EUR') != -1 else row.Salary
  row.Salary = row.Salary.replace('руб.','')
  row.Salary = row.Salary.replace('руб','')
  row.Salary = row.Salary.replace('неуказана','0')
  row.Salary = float(row.Salary) 

  #Experience
  row.Experience = row.Experience.replace('Опыт работы —','')
  row.Experience = row.Experience.replace('Опыт работы','')
  row.Experience = row.Experience.replace('года',';')
  row.Experience = row.Experience.replace('лет',';')
  row.Experience = row.Experience.replace('годов',';')
  row.Experience = row.Experience.replace('год',';')
  row.Experience = row.Experience.replace('года',';')
  row.Experience = row.Experience.replace('месяцев','')
  row.Experience = row.Experience.replace('месяца','')
  row.Experience = row.Experience.replace('месяц','')
  row.Experience = row.Experience.replace('нет','0')
  #Приводим к месяцам, если были года умножаем на 12 иначе берем как есть 
  row.Experience = (int(row.Experience.split(';')[0].strip()) * 12 + (int(row.Experience.split(';')[1].strip())) if row.Experience.split(';')[1].strip() != '' else int(row.Experience.split(';')[0].strip()) * 12) if len(row.Experience.split(';')) >1 else int(row.Experience.strip()) 

  #Schedule
  #вахтовый метод = 0, удаленная работа = 1, гибкий график = 2, полный день = 3, сменный график = 4
  row.Schedule = row.Schedule.replace('вахтовый метод','0')
  row.Schedule = row.Schedule.replace('удаленная работа','1')
  row.Schedule = row.Schedule.replace('гибкий график','2')
  row.Schedule = row.Schedule.replace('полный день','3')
  row.Schedule = row.Schedule.replace('сменный график','4')

  #Employment
  #проектная работа = 0, неполный рабочий день = 1, полный рабочий день = 2, полная занятость = 3, частичная занятость = 4, стажировка = 5, волонтерство = 6
  row.Employment = row.Employment.replace('проектная работа','0')
  row.Employment = row.Employment.replace('неполный рабочий день','1')
  row.Employment = row.Employment.replace('полный рабочий день','2')
  row.Employment = row.Employment.replace('полная занятость','3')
  row.Employment = row.Employment.replace('частичная занятость','4')
  row.Employment = row.Employment.replace('стажировка','5')
  row.Employment = row.Employment.replace('волонтерство','6')

  #Birth
  for i in month_name:
     row.Birth         = row.Birth.replace(month_name[i],'-'+str(i)+'-')
 
  row.Birth         = row.Birth.replace('родился','')
  row.Birth         = row.Birth.replace('родилась','')
  row.Birth = row.Birth.replace(' -','-')
  row.Birth = row.Birth.replace('- ','-')
  row.Birth = row.Birth+' 00:00:00'

  row.Birth = '1-1-1' +row.Birth if row.Birth == ' 00:00:00' else row.Birth
  day = int(row.Birth.split('-')[0].strip()) 
  month = int(row.Birth.split('-')[1].strip()) 
  year = row.Birth.split('-')[2].strip() 
  time = year.split(' ')[1].strip()
  year = int(year.split(' ')[0].strip())

  row.Birth = datetime.datetime(year,month,day).utcnow().timestamp()

  #Update_resume 
  for i in month_name:
    row.Update_resume = row.Update_resume.replace(month_name[i],'-'+str(i)+'-')
 
  row.Update_resume = row.Update_resume.replace('в','')
  row.Update_resume = row.Update_resume.replace(' -','-')
  row.Update_resume = row.Update_resume.replace('- ','-')
  row.Update_resume = row.Update_resume+':00'

  day = int(row.Update_resume.split('-')[0].strip()) 
  month = int(row.Update_resume.split('-')[1].strip()) 
  year = row.Update_resume.split('-')[2].strip() 
  year = year.replace('  ',';')
  year = year.replace(' ',';')
   
  time = year.split(';')[1].strip()
  year = int(year.split(';')[0].strip())
  hour = int(time.split(':')[0].strip())
  minute = int(time.split(':')[1].strip())

  row.Update_resume = datetime.datetime(year,month,day,hour,minute,0).utcnow().timestamp()

  #Response_date
  row.Response_date = datetime.datetime.utcnow().timestamp()

In [201]:
#Переведем в бинарные значения

#проектная работа = 0, неполный рабочий день = 1, полный рабочий день = 2, полная занятость = 3, частичная занятость = 4, стажировка = 5, волонтерство = 6
Employment_0 = data['Employment'].apply(lambda x: 1 if x.find('0') else 0)
Employment_1 = data['Employment'].apply(lambda x: 1 if x.find('1') else 0)
Employment_2 = data['Employment'].apply(lambda x: 1 if x.find('2') else 0)
Employment_3 = data['Employment'].apply(lambda x: 1 if x.find('3') else 0)
Employment_4 = data['Employment'].apply(lambda x: 1 if x.find('4') else 0)
Employment_5 = data['Employment'].apply(lambda x: 1 if x.find('5') else 0)
Employment_6 = data['Employment'].apply(lambda x: 1 if x.find('6') else 0)

data = data.assign(Employment_0=Employment_0,
                    Employment_1=Employment_1,
                    Employment_2=Employment_2,
                    Employment_3=Employment_3,
                    Employment_4=Employment_4,
                    Employment_5=Employment_5,
                    Employment_6=Employment_6)

#вахтовый метод = 0, удаленная работа = 1, гибкий график = 2, полный день = 3, сменный график = 4
Schedule_0 = data['Schedule'].apply(lambda x: 1 if x.find('0') else 0)
Schedule_1 = data['Schedule'].apply(lambda x: 1 if x.find('1') else 0)
Schedule_2 = data['Schedule'].apply(lambda x: 1 if x.find('2') else 0)
Schedule_3 = data['Schedule'].apply(lambda x: 1 if x.find('3') else 0)
Schedule_4 = data['Schedule'].apply(lambda x: 1 if x.find('4') else 0)

data = data.assign(Schedule_0=Schedule_0,
                    Schedule_1=Schedule_1,
                    Schedule_2=Schedule_2,
                    Schedule_3=Schedule_3,
                    Schedule_4=Schedule_4)


In [202]:
data = data.drop(columns=['Schedule', 'Employment'])

In [203]:
#City
tokenizer = Tokenizer(num_words=30, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer.fit_on_texts(data.City) #"скармливаем" наши тексты, т.е даём в обработку методу, который соберет словарь частотности
City_Indexes = tokenizer.texts_to_sequences(data.City)
data = data.assign(City=City_Indexes)

In [204]:
#Full_name
tokenizer = Tokenizer(num_words=100, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer.fit_on_texts(data.Full_name)

Full_name_Indexes = tokenizer.texts_to_sequences(data.Full_name) 
data = data.assign(Full_name=Full_name_Indexes)

In [205]:
#Education
tokenizer = Tokenizer(num_words=200, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer.fit_on_texts(data.Education) #"скармливаем" наши тексты, т.е даём в обработку методу, который соберет словарь частотности

Education_Indexes = tokenizer.texts_to_sequences(data.Education) 
data = data.assign(Education=Education_Indexes)

In [206]:
#Job
tokenizer = Tokenizer(num_words=50, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer.fit_on_texts(data.Job) #"скармливаем" наши тексты, т.е даём в обработку методу, который соберет словарь частотности

Job_Indexes = tokenizer.texts_to_sequences(data.Job) 
data = data.assign(Job=Job_Indexes)

In [207]:
#Last_place
tokenizer = Tokenizer(num_words=100, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer.fit_on_texts(data.Last_place) #"скармливаем" наши тексты, т.е даём в обработку методу, который соберет словарь частотности

Last_place_Indexes = tokenizer.texts_to_sequences(data.Last_place) 
data = data.assign(Last_place=Last_place_Indexes)

In [208]:
#Position
tokenizer = Tokenizer(num_words=20, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer.fit_on_texts(data.Position) #"скармливаем" наши тексты, т.е даём в обработку методу, который соберет словарь частотности

Position_Indexes = tokenizer.texts_to_sequences(data.Position) 
data = data.assign(Position=Position_Indexes)

In [209]:
#Suitable
#Обработаем отдельно колонку с ответами
for i, row in data.iterrows():
   row.Suitable = 1 if str(row.Suitable).find('да') != -1 else 0

In [210]:
#Смотрим что получилось
data.max_columns = 30
data.head(-15)

,Response_date,Full_name,Gender,Salary,Job,City,Experience,Last_place,Position,Education,...,Employment_2,Employment_3,Employment_4,Employment_5,Employment_6,Schedule_0,Schedule_1,Schedule_2,Schedule_3,Schedule_4
0,1684078617.371898,"[1, 1]",0,90000.0,"[4, 3, 2]","[1, 7, 6, 2, 5, 7, 6, 2, 4]",70,"[1, 1, 64, 28]","[4, 5, 1]","[3, 76, 1, 43, 2, 50, 71, 103, 6]",...,1,1,1,1,1,1,0,1,1,1
1,1684078617.373823,"[1, 11, 50]",0,0.0,"[4, 3, 2]","[9, 6, 2, 5, 6, 2, 4]",143,"[4, 1]","[2, 10, 1, 1, 11, 1]","[3, 17, 1, 33, 16, 1, 1, 12, 6]",...,1,0,1,1,1,1,0,1,1,1
2,1684078617.375834,"[1, 15, 39]",1,150000.0,"[4, 3, 2]","[11, 12, 3, 2, 5, 9, 3, 2, 4]",86,"[1, 35]","[7, 5, 1, 11, 1, 4]","[3, 27, 13, 41, 4, 2, 1, 146, 7, 118, 13, 26]",...,1,1,1,1,1,1,0,1,1,1
3,1684078617.377755,"[1, 1, 1]",1,0.0,"[4, 3, 2]","[9, 13, 1, 14, 15, 3, 2, 4]",106,[1],"[2, 3, 1, 1]","[34, 43, 16, 4, 10, 2, 8, 36, 119, 120, 6, 30,...",...,1,1,1,0,1,1,0,1,1,1
4,1684078617.379663,"[1, 40, 39]",1,150000.0,"[4, 3, 2]","[1, 14, 15, 3, 2, 4]",69,"[5, 9, 10, 11, 6]","[2, 15, 3]","[3, 24, 147, 15, 107, 108, 7, 35, 37, 22, 52, ...",...,1,1,1,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,1684078618.450335,"[1, 8, 4]",1,0.0,"[3, 2, 5, 6, 7]","[9, 3, 2, 5, 3, 2, 10, 4]",184,"[1, 1]","[1, 1]","[3, 42, 13, 41, 4, 77, 2, 13, 26]",...,1,1,1,1,1,1,0,1,1,1
563,1684078618.45222,"[1, 1]",1,80000.0,"[3, 2, 5, 6, 7]","[19, 3, 2, 5, 7, 3, 2, 4]",25,"[1, 1]","[2, 3]","[3, 23, 1, 1, 2, 169]",...,1,1,1,1,1,1,0,1,1,1
564,1684078618.454067,"[1, 1]",0,250000.0,"[3, 2, 5, 6, 7]","[9, 7, 6, 2, 5, 6, 2, 4]",45,[1],"[2, 3]","[28, 17, 77, 2]",...,1,1,1,1,1,1,0,1,1,1
565,1684078618.455896,"[1, 1, 4]",1,120000.0,"[3, 2, 5, 6, 7]","[23, 7, 3, 2, 5, 3, 2, 4]",69,"[1, 24]","[6, 19]","[3, 24, 75, 4, 2, 196, 197]",...,1,1,0,1,1,1,0,1,1,1


In [211]:
#Список ответов делаем
answers = data.Suitable.to_numpy()
data = data.drop(columns=['Suitable'])


In [151]:
data.to_csv('data.csv')

In [300]:
Full_nameMaxI = 0
PositionMaxI = 0
Last_placeMaxI = 0
JobMaxI = 0
EducationMaxI = 0
CityMaxI = 0
for i, row in data.iterrows():
    Full_nameMaxI   = max(Full_nameMaxI,len(row.Full_name))
    PositionMaxI    = max(PositionMaxI,len(row.Position))
    Last_placeMaxI  = max(Last_placeMaxI,len(row.Last_place))
    JobMaxI         = max(JobMaxI,len(row.Job))
    EducationMaxI   = max(EducationMaxI,len(row.Education))
    CityMaxI        = max(CityMaxI,len(row.City))


In [309]:
#Приведим все списки к однообразию
for i, row in data.iterrows():
      row.Full_name     = row.Full_name+[0 for x in range(Full_nameMaxI-len(row.Full_name))]
      print(row.Full_name)
      row.Position      = row.Position+[0 for x in range(PositionMaxI-len(row.Position))]
      row.Last_place    = row.Last_place+[0 for x in range(Last_placeMaxI-len(row.Last_place))]
      row.Job           = row.Job+[0 for x in range(JobMaxI-len(row.Job))]
      row.Education     = row.Education+[0 for x in range(EducationMaxI-len(row.Education))]
      row.City          = row.City+[0 for x in range(CityMaxI-len(row.City))]



[1, 1, 0, 0, 0]
[1, 11, 50, 0, 0]
[1, 15, 39, 0, 0]
[1, 1, 1, 0, 0]
[1, 40, 39, 0, 0]
[1, 1, 0, 0, 0]
[1, 30, 0, 0, 0]
[1, 22, 1, 0, 0]
[1, 51, 0, 0, 0]
[41, 12, 1, 0, 0]
[1, 1, 1, 0, 0]
[78, 79, 0, 0, 0]
[52, 18, 16, 0, 0]
[1, 42, 53, 0, 0]
[1, 1, 54, 0, 0]
[1, 1, 1, 0, 0]
[1, 1, 0, 0, 0]
[1, 1, 1, 0, 0]
[1, 1, 1, 10, 1]
[1, 13, 17, 0, 0]
[1, 10, 1, 0, 0]
[1, 80, 1, 0, 0]
[1, 1, 1, 0, 0]
[1, 81, 1, 0, 0]
[1, 13, 0, 0, 0]
[1, 82, 0, 0, 0]
[1, 1, 1, 0, 0]
[1, 1, 0, 0, 0]
[1, 55, 1, 0, 0]
[1, 1, 1, 0, 0]
[23, 24, 25, 0, 0]
[1, 1, 1, 0, 0]
[1, 56, 0, 0, 0]
[1, 13, 0, 0, 0]
[1, 57, 0, 0, 0]
[83, 2, 3, 0, 0]
[1, 19, 0, 0, 0]
[84, 31, 3, 0, 0]
[1, 1, 26, 0, 0]
[85, 20, 86, 0, 0]
[87, 5, 27, 0, 0]
[1, 88, 32, 0, 0]
[89, 33, 58, 0, 0]
[90, 5, 0, 0, 0]
[91, 59, 60, 0, 0]
[1, 11, 0, 0, 0]
[1, 1, 1, 0, 0]
[1, 81, 1, 0, 0]
[1, 15, 0, 0, 0]
[1, 13, 1, 0, 0]
[1, 92, 0, 0, 0]
[1, 8, 0, 0, 0]
[1, 42, 1, 0, 0]
[1, 1, 0, 0, 0]
[1, 82, 1, 0, 0]
[1, 1, 0, 0, 0]
[1, 1, 0, 0, 0]
[1, 1, 0, 0, 0]
[1, 1, 0, 0,

In [308]:
data.to_csv('data.csv')

In [307]:
dataList = [] 

for i, row in data.iterrows():
      row_data = np.hstack(row)
      print(row_data.shape)
      dataList.append(row_data)

dataset = np.array(dataList)

print(dataset.shape)

(49,)
(49,)
(54,)
(59,)
(52,)
(50,)
(47,)
(40,)
(58,)
(42,)
(52,)
(45,)
(76,)
(48,)
(53,)
(67,)
(46,)
(58,)
(74,)
(53,)
(49,)
(47,)
(52,)
(51,)
(57,)
(54,)
(52,)
(48,)
(75,)
(69,)
(51,)
(61,)
(57,)
(53,)
(52,)
(52,)
(47,)
(46,)
(61,)
(45,)
(73,)
(49,)
(48,)
(43,)
(49,)
(52,)
(70,)
(57,)
(52,)
(57,)
(43,)
(65,)
(55,)
(42,)
(46,)
(42,)
(51,)
(50,)
(59,)
(65,)
(59,)
(45,)
(55,)
(46,)
(41,)
(55,)
(56,)
(69,)
(50,)
(43,)
(45,)
(61,)
(52,)
(64,)
(48,)
(54,)
(64,)
(47,)
(44,)
(54,)
(54,)
(54,)
(46,)
(57,)
(44,)
(51,)
(51,)
(43,)
(46,)
(46,)
(50,)
(47,)
(48,)
(51,)
(40,)
(57,)
(67,)
(45,)
(49,)
(44,)
(56,)
(49,)
(45,)
(44,)
(67,)
(44,)
(50,)
(56,)
(46,)
(46,)
(47,)
(44,)
(55,)
(58,)
(44,)
(46,)
(43,)
(74,)
(53,)
(48,)
(53,)
(58,)
(44,)
(52,)
(50,)
(46,)
(45,)
(50,)
(44,)
(47,)
(51,)
(45,)
(43,)
(53,)
(82,)
(44,)
(59,)
(41,)
(49,)
(71,)
(46,)
(54,)
(55,)
(48,)
(47,)
(43,)
(63,)
(45,)
(44,)
(71,)
(44,)
(55,)
(42,)
(50,)
(49,)
(53,)
(46,)
(45,)
(61,)
(60,)
(51,)
(47,)
(55,)
(46,)
(50,)
(53,)
(53,

<ipython-input-307-d7a49fdf2a2a>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array(dataList)


In [102]:
x_train, x_test, y_train, y_test = train_test_split(dataset, # набор параметров
                                                    answers, # набор меток классов
                                                    test_size=0.2, # процент в тестовую
                                                    shuffle=True, #  перемешивание
                                                    random_state=3) # воспроизводимость

In [106]:
y_train.shape

(463,)

In [104]:
# Оптимизаторы
from tensorflow.keras.optimizers import Adam

# Создание последовательной модели
model = Sequential()
model.add(Dropout(30, input_shape=(x_train.shape[1],)))  
model.add(Dense(60, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=0.001), 
              metrics=['accuracy'])

# Обучение сети
history = model.fit(x_train, 
                    y_train, 
                    batch_size=8, 
                    epochs=200,
                    validation_split=0.2, 
                    verbose=1)

ValueError: ignored

In [ ]:
# Обучение нейронной сети
history = model.fit(x_train,               # Обучающая выборка параметров
          y_train,               # Обучающая выборка меток класса
          batch_size=8,          # Размер батча (пакета)
          epochs=100,            # Количество эпох обучения
          validation_split=0.2,  # Доля проверочной выборки
          verbose=1)             # Отображение хода обучения

In [ ]:
# Визуализация точности на обучающей выборке
plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

plt.plot(history.history['loss'], 
         label='Ошибка на обучающем наборе')
plt.plot(history.history['val_loss'], 
         label='Ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')
plt.legend()
plt.show()